# Exercise 2: use stocks.csv file for the this exercise
1. Load the stocks.csv file in a new Rdd
2. Find stocks records in 2016
3. Find stocks top 10 records based on the trading volume in 2016
4. Find average volume per stock traded in 2016
5. Find top 10 stocks based on highest trading volume in 2016

In [2]:
from pyspark import SparkContext,SparkConf
sc = SparkContext.getOrCreate()

Load the stocks.csv file in a new Rdd

In [6]:
stocks = sc.textFile("stocks.csv")

In [7]:
stocks.count()

326456

In [8]:
stocks.first()

'date,open,high,low,close,volume,adjclose,symbol'

In [9]:
stocks.cache()

stocks.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

Find stocks records in 2016

In [10]:
stocks2016 = stocks.filter(lambda line: line.startswith("2016"))
stocks2016.count()

78041

In [11]:
stocks2016.first()

'2016-01-04,46.119999,46.130001,45.360001000000004,45.799999,3472200.0,44.870315000000005,XLNX'

In [12]:
stocks2016.sortBy(lambda line: line.split(",")[5], False).take(10)

['2016-01-26,61.740002000000004,62.509997999999996,61.509997999999996,62.349998,999900.0,61.430663,AWK',
 '2016-08-03,75.599998,76.099998,75.559998,76.089996,999900.0,75.520442,KLAC',
 '2016-05-23,12.48,12.56,12.31,12.48,9997600.0,12.307377,KEY',
 '2016-02-22,40.139998999999996,40.93,40.07,40.84,9997300.0,40.60075,AAL',
 '2016-07-12,38.470001,38.91,38.240002000000004,38.790001000000004,9997200.0,38.790001000000004,PYPL',
 '2016-04-11,104.059998,104.059998,102.300003,102.620003,999700.0,102.350541,INTU',
 '2016-02-01,48.939999,49.990002000000004,48.790001000000004,49.900002,9996700.0,48.283396,SO',
 '2016-06-24,41.869999,42.73,40.630001,40.970001,9996400.0,40.970001,VIAB',
 '2016-01-04,117.25,117.730003,115.870003,117.58000200000001,9995000.0,115.84442800000001,MCD',
 '2016-02-23,131.28999299999998,131.91000400000001,130.91000400000001,131.529999,999500.0,129.68786599999999,KMB']

Find average volume per stock traded in 2016

In [13]:
stocks2016.map(lambda line: line.split(","))\
.map(lambda tokens: (tokens[7], float(tokens[5])))\
.first()

('XLNX', 3472200.0)

In [15]:
stocks2016.map(lambda line: line.split(","))\
.map(lambda tokens: (tokens[7], float(tokens[5])))\
.groupByKey().mapValues(lambda volumes: sum(volumes)/ len(volumes))\
.first()

('XLNX', 2872973.717948718)

In [16]:
top10 = stocks2016.map(lambda line: line.split(","))\
.map(lambda tokens: (tokens[7], float(tokens[5])))\
.groupByKey().mapValues(lambda volumes: sum(volumes)/ len(volumes))\
.sortBy(lambda p: p[1], False)\
.take(10)

for r in top10:
    print(r)

('BAC', 109953689.74358974)
('FCX', 47979558.333333336)
('CHK', 41622735.256410256)
('AAPL', 40944183.974358976)
('GE', 37751663.461538464)
('F', 37432197.43589743)
('PFE', 35777183.974358976)
('MSFT', 34194448.07692308)
('FB', 28902566.025641024)
('MU', 27260807.692307692)


In [17]:
stocksData = stocks.filter(lambda line: not line.startswith("date"))
stocksData.first()

'2014-01-02,46.68,46.700001,45.900002,45.970001,3436800.0,42.9403,XLNX'

# Parse the date field as datetime object

In [18]:
from datetime import datetime
d = datetime.strptime("2016-07-17", "%Y-%m-%d")
print(type(d), d.year)

<class 'datetime.datetime'> 2016


In [19]:
stocksData.filter(lambda line: datetime.strptime(line.split(",")[0],"%Y-%m-%d").year == 2016).first()

'2016-01-04,46.119999,46.130001,45.360001000000004,45.799999,3472200.0,44.870315000000005,XLNX'